In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from sklearn.preprocessing import MinMaxScaler

In [2]:
seed = 13
tf.random.set_seed(seed)

In [3]:
data = pd.read_csv('..\\Data\\ze41_mol_desc_db_red.csv', header=0, sep=';', decimal=',')

In [4]:
col_names = data.columns
X = data[col_names[3:]].astype('float32')
y = data[col_names[2]].astype('float32')

In [5]:
X_scaled = MinMaxScaler(feature_range=(0,1)).fit_transform(X)
X_scaled = pd.DataFrame(X_scaled)
X_scaled.columns = X.columns

In [6]:
class Autoencoder(keras.models.Model):
    def __init__(self, latent_dim=5):
        super(Autoencoder, self).__init__()
        self.latent_dim = latent_dim   
        self.encoder = tf.keras.Sequential([
          keras.layers.Dense(latent_dim, activation='relu'),
        ])
        self.decoder = tf.keras.Sequential([
          keras.layers.Dense(len(X_scaled.columns), activation='linear')
        ])
    
    def call(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return decoded

In [7]:
autoencoder = Autoencoder(latent_dim=5)

In [8]:
autoencoder.compile(
    optimizer=tf.optimizers.Adam(learning_rate=0.005),
    loss='mean_squared_error')

In [9]:
autoencoder.fit(X_scaled, X_scaled, validation_split=0.2, epochs=100)

Epoch 1/100
2/2 [==============================] - 0s 62ms/step - loss: 0.2240 - val_loss: 0.1959
Epoch 2/100
2/2 [==============================] - 0s 8ms/step - loss: 0.2128 - val_loss: 0.1755
Epoch 3/100
2/2 [==============================] - 0s 8ms/step - loss: 0.1881 - val_loss: 0.1555
Epoch 4/100
2/2 [==============================] - 0s 8ms/step - loss: 0.1633 - val_loss: 0.1312
Epoch 5/100
2/2 [==============================] - 0s 8ms/step - loss: 0.1354 - val_loss: 0.1098
Epoch 6/100
2/2 [==============================] - 0s 8ms/step - loss: 0.1121 - val_loss: 0.0922
Epoch 7/100
2/2 [==============================] - 0s 8ms/step - loss: 0.0927 - val_loss: 0.0806
Epoch 8/100
2/2 [==============================] - 0s 8ms/step - loss: 0.0793 - val_loss: 0.0700
Epoch 9/100
2/2 [==============================] - 0s 8ms/step - loss: 0.0681 - val_loss: 0.0634
Epoch 10/100
2/2 [==============================] - 0s 8ms/step - loss: 0.0615 - val_loss: 0.0591
Epoch 11/100
2/2 [==========

2/2 [==============================] - 0s 8ms/step - loss: 0.0440 - val_loss: 0.0460
Epoch 85/100
2/2 [==============================] - 0s 8ms/step - loss: 0.0440 - val_loss: 0.0460
Epoch 86/100
2/2 [==============================] - 0s 8ms/step - loss: 0.0440 - val_loss: 0.0459
Epoch 87/100
2/2 [==============================] - 0s 8ms/step - loss: 0.0439 - val_loss: 0.0459
Epoch 88/100
2/2 [==============================] - 0s 8ms/step - loss: 0.0439 - val_loss: 0.0458
Epoch 89/100
2/2 [==============================] - 0s 8ms/step - loss: 0.0438 - val_loss: 0.0456
Epoch 90/100
2/2 [==============================] - 0s 8ms/step - loss: 0.0437 - val_loss: 0.0456
Epoch 91/100
2/2 [==============================] - 0s 8ms/step - loss: 0.0437 - val_loss: 0.0454
Epoch 92/100
2/2 [==============================] - 0s 8ms/step - loss: 0.0436 - val_loss: 0.0453
Epoch 93/100
2/2 [==============================] - 0s 8ms/step - loss: 0.0436 - val_loss: 0.0452
Epoch 94/100
2/2 [===============

In [10]:
best_cols = ['P_VSA_MR_5', 'Mor04m', 'E1p', 'CATS3D_02_AP', 'LUMO / eV']

In [11]:
X_sel = X_scaled[best_cols]

In [12]:
encoded = autoencoder.encoder(X_scaled.to_numpy()).numpy()

In [14]:
for i in range(5):
    print(sorted(X_sel.to_numpy()[i,:]), sorted(encoded[i, :]))

[0.14643973, 0.27012992, 0.2947153, 0.68376565, 1.0] [0.0, 0.0, 0.0, 0.0, 3.03231]
[0.0, 0.09568678, 0.14805192, 0.47917604, 0.63227195] [0.0, 0.0, 0.0, 0.0, 4.256732]
[0.0, 0.06753248, 0.09568678, 0.3123034, 0.67890733] [0.0, 0.0, 0.0, 0.0, 5.125967]
[0.0, 0.0, 0.06753248, 0.40683958, 0.6027766] [0.0, 0.0, 0.0, 0.0, 5.3694844]
[0.0, 0.0, 0.0, 0.3791273, 0.48096734] [0.0, 0.0, 0.0, 0.0, 4.650193]
